Simple product state

In [69]:
using ITensors
using ITensorMPS

# Define the Hadamard gate
function hadamard(s)
    sx = op("Sx", s)
    sz = op("Sz", s)

    h = (sx + sz) * sqrt(2.0)
    
    return h
end


N = 5 #number of qubits
sites = siteinds("S=1/2", N)
psi0 = productMPS(sites, "Up") #all in the up-state (|0>)


ampo = AutoMPO()
for j in 1:N
  ampo += 2.0, "Sx", j                  # add an Sx on site j
end
mpoX = MPO(ampo, sites)            # convert your sum into an MPO
val = inner(psi0', mpoX, psi0)    # contracts bra ψ, mpoX, ket ψ
println("⟨X₁ + X₂ + ⋯ + X_N⟩ = ", real(val))

product_tensor = ITensor(1)
for j in 1:N
    s = sites[j]
    product_tensor *= 2 * op("Sx", s)
end
# Create an MPO from the product tensor
mpoSigmaX = MPO(product_tensor, sites)
val = inner(psi0', mpoSigmaX, psi0)
println("⟨X₁ * X₂ * ⋯ * X_N⟩ = ", real(val))


In [70]:
# Apply Hadamard to site 3
site_index = 3
h = hadamard(sites[site_index])

# Apply the gate to the MPS - use an array for the site index
psi = apply(h, psi0, [site_index])  # Note the [site_index] array syntax here

# Normalize the state
psi = normalize(psi)

# Verify by computing expectation values
for j in 1:N
    # Create a simple MPO for each measurement
    ampo_x = AutoMPO()
    ampo_x += 2.0, "Sx", j
    mpo_x = MPO(ampo_x, sites)
    
    ampo_z = AutoMPO()
    ampo_z += 2.0, "Sz", j
    mpo_z = MPO(ampo_z, sites)
    
    expval_x = real(inner(psi', mpo_x, psi))
    expval_z = real(inner(psi', mpo_z, psi))
    
    println("Site $j: ⟨σx⟩ = $(round(expval_x, digits=4)), ⟨σz⟩ = $(round(expval_z, digits=4))")
end

println("⟨X₁ + X₂ + ⋯ + X_N⟩ = $(round(real(inner(psi', mpoX, psi)), digits=4))")
println("⟨X₁ * X₂ * ⋯ * X_N⟩ = $(round(real(inner(psi', mpoSigmaX, psi)), digits=4))")


In [71]:
sites_to_apply = [1, 2, 3, 4, 5]
psi = psi0
for site_index in sites_to_apply
    h = hadamard(sites[site_index])
    psi = apply(h, psi, [site_index])  # Note the brackets
    psi = normalize(psi)
end

# Verify by computing expectation values
for j in 1:N
    # Create a simple MPO for each measurement
    ampo_x = AutoMPO()
    ampo_x += 2.0, "Sx", j
    mpo_x = MPO(ampo_x, sites)
    
    ampo_z = AutoMPO()
    ampo_z += 2.0, "Sz", j
    mpo_z = MPO(ampo_z, sites)
    
    expval_x = real(inner(psi', mpo_x, psi))
    expval_z = real(inner(psi', mpo_z, psi))
    
    println("Site $j: ⟨σx⟩ = $(round(expval_x, digits=4)), ⟨σz⟩ = $(round(expval_z, digits=4))")
end

println("⟨X₁ + X₂ + ⋯ + X_N⟩ = $(round(real(inner(psi', mpoX, psi)), digits=4))")
println("⟨X₁ * X₂ * ⋯ * X_N⟩ = $(round(real(inner(psi', mpoSigmaX, psi)), digits=4))")

Two-qubit unitary

In [72]:
using ITensors
using ITensorMPS
using Optim
using LinearAlgebra
using Printf
using Random

# Define basic operators
function pauli_operators(s)
    sigmax = 2 * op("Sx", s)
    sigmay = 2 * op("Sy", s)
    sigmaz = 2 * op("Sz", s)
    id = op("Id", s)
    return [id, sigmax, sigmay, sigmaz]  # σ⁰, σˣ, σʸ, σᶻ
end

# Define the Hadamard gate
function hadamard(s)
    sx = op("Sx", s)
    sz = op("Sz", s)
    h = (sx + sz) * sqrt(2.0)
    return h
end

N = 5 #number of qubits
sites = siteinds("S=1/2", N)
psi0 = productMPS(sites, "Up") #all in the up-state (|0>)
#act with Hadamard on the first site
site_index = 1
h = hadamard(sites[site_index])
# Apply the gate to the MPS - use an array for the site index
psi0 = apply(h, psi0, [site_index])  # Note the [site_index] array syntax here
psi0 = normalize(psi0);

# Create parameterized two-qubit unitary, acting on sites i and i + 1
function create_unitary(sites, i, params)
    # params has 15 parameters that encode the unitary
    
    s1 = sites[i]
    s2 = sites[i+1]
    
    # Get Pauli operators for both sites
    ops1 = pauli_operators(s1)  # [I, σx, σy, σz] for site 1
    ops2 = pauli_operators(s2)  # [I, σx, σy, σz] for site 2
    
    # Initialize the generator of the unitary (Hermitian operator)
    generator = 0.0 * ops1[1] * ops2[1]#pick custom phase 
    
    # Two-qubit σᵃ⊗σᵇ terms
    param_idx = 1
    for a in 2:4, b in 2:4
        term = ops1[a] * ops2[b]
        generator += -im * params[param_idx] * term
        param_idx += 1
    end

    #Single-qubit σᵃ⊗I terms
    for a in 2:4
        term = ops1[a] * ops2[1]
        generator += -im * params[param_idx] * term
        param_idx += 1
    end

    #Single-qubit I⊗σᵇ terms
    for b in 2:4
        term = ops1[1] * ops2[b]
        generator += -im * params[param_idx] * term
        param_idx += 1
    end

    return exp(generator)
end

# Apply the ladder circuit to a given MPS psi
function apply_circuit(psi, sites, params)
    N = length(sites)
    psi_evolved = deepcopy(psi)
    # Apply unitaries sequentially: (1,2), (2,3), ...
    for i in 1:(N-1)
        U = create_unitary(sites, i, params)
        psi_evolved = apply(U, psi_evolved, [i, i+1])
        normalize!(psi_evolved)
    end   
    return psi_evolved
end

function cost_function(psi, sites, g)
    N = length(sites)

    X_string = ITensor(1)
    for j in 1:N
        s = sites[j]
        X_string *= 2 * op("Sx", s)
    end
    X_string = MPO(X_string, sites)

    O_string = ITensor(1)
    O_string *= 2 * op("Sz", sites[1])
    O_string *= 2 * op("Sy", sites[2])
    for j in 3:(N - 2)
        s = sites[j]
        O_string *= 2 * op("Sx", s)
    end
    O_string *= 2 * op("Sy", sites[N - 1])
    O_string *= 2 * op("Sz", sites[N])
    O_string = MPO(O_string, sites)


    ampo = AutoMPO()
    # Add Z_j * Z_{j+1} terms for each neighboring pair
    for j in 1:(N-1)
        ampo .+= 4.0, "Sz", j, "Sz", j + 1
    end
    ZZ_term = MPO(ampo, sites)

    return -real(inner(psi', X_string, psi)) - 0.5 * (1 - g) * real(inner(psi', O_string, psi)) - 0.5 * (1 + g) * real(inner(psi', ZZ_term, psi)) / (N - 1)
end

function cost_function_params(params, psi0, sites, g)
    psi = apply_circuit(psi0, sites, params)
    return cost_function(psi, sites, g)
end

# Function to optimize the parameters
function optimize_circuit(psi0, sites, g, initial_params; max_iter=1000)
    N = length(sites)
    
    # Create the objective function for optimization
    function objective(p)
        cost = cost_function_params(p, psi0, sites, g)
        # Print progress
        # if rand() < 0.1  # Only print occasionally to avoid too much output
        #     println("Current cost: $cost")
        # end
        return cost
    end
    
    # Run the optimization
    println("Starting optimization...")
    result = optimize(
        objective,
        initial_params,
        BFGS(),
        Optim.Options(iterations=max_iter, show_trace=true)
    )
    
    # Get optimized parameters
    opt_params = Optim.minimizer(result)
    
    # Apply optimized parameters to get final state
    final_psi = apply_circuit(psi0, sites, opt_params)
    
    final_cost = cost_function(final_psi, sites, g)
    
    println("Optimization complete!")
    println("Final cost: $final_cost")

    return opt_params
end

# params = 0.01 * randn(15)
params_GHZ = zeros(15)
params_GHZ[7] = pi / 4
params_GHZ[12] = - pi / 4
params_GHZ[13] = - pi / 4

params = params_GHZ
g = 1

final_params = optimize_circuit(psi0, sites, g, params)


15-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.7853981633974483
  0.0
  0.0
  0.0
  0.0
 -0.7853981633974483
 -0.7853981633974483
  0.0
  0.0

Fidelity with respect to (1) GHZ state, (2) cluster state, and (3) product state |+...+>

In [ ]:
psi = apply_circuit(psi0, sites, final_params)

# Create |00...0> state
state_0 = productMPS(sites, "Up")
# Create |11...1> state
state_1 = productMPS(sites, "Dn")
# Create the superposition: (|00...0> + |11...1>)/sqrt(2)
ghz_state = add(state_0, state_1)
normalize!(ghz_state)

fidelity_ghz = abs2(inner(psi', ghz_state))
println("Fidelity wrt to GHZ = $(round(fidelity_ghz, digits=4))")

